In [28]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, mean_absolute_error
import numpy as np
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.metrics import Metric
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import TransformedTargetRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
import time

import warnings
warnings.filterwarnings("ignore", message="'pin_memory' argument is set as true but not supported on MPS")

In [2]:
df = pd.read_csv('final_data.csv', low_memory=False)
df.salary_gross.fillna(False, inplace=True)
df.shape

/var/folders/7m/n6pdmsh950scyftx2z0wsv3w0000gn/T/ipykernel_6411/990545542.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.salary_gross.fillna(False, inplace=True)
/var/folders/7m/n6pdmsh950scyftx2z0wsv3w0000gn/T/ipykernel_6411/990545542.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.salary_gross.

(709524, 43)

In [3]:
def culc_metrics(y_test, y_pred):
    test_mse = mean_squared_error(y_test, y_pred)
    rmse = test_mse**0.5
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    def symmetric_mean_absolute_percentage_error(y_true, y_pred):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        smape = 100 * np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))
        return smape

    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)

    medae = median_absolute_error(y_test, y_pred)

    print(f'Корень из среднеквадратичной ошибки (RMSE): {rmse}')
    print(f"R² Score: {r2}")
    print(f"Средняя абсолютная ошибка (MAE): {mae}")
    print(f"Средняя абсолютная процентная ошибка (SMAPE): {smape:.2f}%")
    print(f"Медианная абсолютная ошибка (MedAE): {medae}")

In [4]:
cat_columns = ['premium', 'has_test', 'response_letter_required', 'area_name', 'salary_currency', 'salary_gross', 'type_name', 'address_city', 'address_metro_station_name', 'address_metro_line_name', 'address_metro_stations_0_line_name', 'archived', 'employer_name', 'employer_accredited_it_employer', 'employer_trusted', 'schedule_name', 'accept_temporary', 'professional_roles_0_name', 'accept_incomplete_resumes', 'experience_name', 'employment_name', 'address_metro_stations_3_station_name', 'address_metro_stations_3_line_name', 'working_time_intervals_0_name', 'working_time_modes_0_name', 'working_days_0_name', 'branding_type', 'branding_tariff', 'department_name', 'insider_interview_id', 'brand_snippet_logo', 'brand_snippet_picture', 'brand_snippet_background_color', 'brand_snippet_background_gradient_angle', 'brand_snippet_background_gradient_color_list_0_position', 'brand_snippet_background_gradient_color_list_1_position', 'category']
text_columns = ['name', 'snippet_requirement', 'snippet_responsibility']
num_columns = ['name_length', 'length']

In [21]:
scaler = StandardScaler()
num_df = pd.DataFrame(scaler.fit_transform(df[num_columns]), columns=num_columns)

In [22]:
label_columns = []
ohe_columns = []

for column in cat_columns:
    if df[column].nunique() > 10:
        label_columns.append(column)
    else:
        ohe_columns.append(column)

to_bool = list(df[cat_columns].select_dtypes(include=['bool']).columns)
df[['salary_gross', 'employer_accredited_it_employer']] = df[['salary_gross', 'employer_accredited_it_employer']].astype(bool).astype(int)
df[to_bool] = df[to_bool].astype(int)

ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe_encoded = ohe.fit_transform(df[ohe_columns])
ohe_feature_names = ohe.get_feature_names_out(ohe_columns).tolist()
encoded_ohe_data = pd.DataFrame(ohe_encoded, columns=ohe_feature_names)

embedding_dim = 5
embeddings = {}

for col in label_columns:
    unique_values = df[col].unique()
    value_to_idx = {v: i for i, v in enumerate(unique_values)}
    df[col+'_idx'] = df[col].map(value_to_idx)

    num_embeddings = len(unique_values)
    embedding_layer = nn.Embedding(num_embeddings, embedding_dim)

    embeddings[col] = {
        'value_to_idx': value_to_idx,
        'embedding': embedding_layer,
        'num_embeddings': num_embeddings
    }

embedded_data = []
for col in label_columns:
    indices = torch.tensor(df[col+'_idx'].values, dtype=torch.long)
    embedded = embeddings[col]['embedding'](indices).detach().numpy()
    embedded_cols = [f"{col}_embed_{i}" for i in range(embedding_dim)]
    embedded_df = pd.DataFrame(embedded, columns=embedded_cols)
    embedded_data.append(embedded_df)

embedded_data = pd.concat(embedded_data, axis=1)
final_data = pd.concat([encoded_ohe_data, embedded_data], axis=1)

for col in label_columns:
    df.drop(col+'_idx', axis=1, inplace=True)

In [7]:
final_data.shape

(709524, 111)

In [8]:
final_data

,premium_1,has_test_1,response_letter_required_1,salary_currency_BYR,salary_currency_EUR,salary_currency_GEL,salary_currency_KGS,salary_currency_KZT,salary_currency_RUR,salary_currency_USD,salary_currency_UZS,salary_gross_1,type_name_Закрытая,type_name_Открытая,type_name_Рекламная,archived_1,employer_trusted_1,schedule_name_Гибкий график,schedule_name_Полный день,schedule_name_Сменный график,schedule_name_Удаленная работа,accept_temporary_1,accept_incomplete_resumes_1,experience_name_Нет опыта,experience_name_От 1 года до 3 лет,experience_name_От 3 до 6 лет,employment_name_Полная занятость,employment_name_Проектная работа,employment_name_Стажировка,employment_name_Частичная занятость,working_time_intervals_0_name_Можно сменами по 4-6 часов в день,working_time_modes_0_name_С началом дня после 16:00,working_days_0_name_По субботам и воскресеньям,branding_type_MAKEUP,branding_type_Unknown,branding_tariff_Unknown,insider_interview_id_1,brand_snippet_logo_Unknown,brand_snippet_picture_Unknown,brand_snippet_background_color_#EF3124,brand_snippet_background_color_#FF5B29,brand_snippet_background_color_Unknown,brand_snippet_background_gradient_angle_134.0,brand_snippet_background_gradient_angle_200.0,brand_snippet_background_gradient_angle_206.43,brand_snippet_background_gradient_angle_67.0,brand_snippet_background_gradient_angle_Unknown,brand_snippet_background_gradient_color_list_0_position_0.0,brand_snippet_background_gradient_color_list_0_position_0.52,brand_snippet_background_gradient_color_list_0_position_6.96,brand_snippet_background_gradient_color_list_0_position_Unknown,brand_snippet_background_gradient_color_list_1_position_40.0,brand_snippet_background_gradient_color_list_1_position_88.86,brand_snippet_background_gradient_color_list_1_position_90.95,brand_snippet_background_gradient_color_list_1_position_94.48,brand_snippet_background_gradient_color_list_1_position_Unknown,area_name_embed_0,area_name_embed_1,area_name_embed_2,area_name_embed_3,area_name_embed_4,address_city_embed_0,address_city_embed_1,address_city_embed_2,address_city_embed_3,address_city_embed_4,address_metro_station_name_embed_0,address_metro_station_name_embed_1,address_metro_station_name_embed_2,address_metro_station_name_embed_3,address_metro_station_name_embed_4,address_metro_line_name_embed_0,address_metro_line_name_embed_1,address_metro_line_name_embed_2,address_metro_line_name_embed_3,address_metro_line_name_embed_4,address_metro_stations_0_line_name_embed_0,address_metro_stations_0_line_name_embed_1,address_metro_stations_0_line_name_embed_2,address_metro_stations_0_line_name_embed_3,address_metro_stations_0_line_name_embed_4,employer_name_embed_0,employer_name_embed_1,employer_name_embed_2,employer_name_embed_3,employer_name_embed_4,professional_roles_0_name_embed_0,professional_roles_0_name_embed_1,professional_roles_0_name_embed_2,professional_roles_0_name_embed_3,professional_roles_0_name_embed_4,address_metro_stations_3_station_name_embed_0,address_metro_stations_3_station_name_embed_1,address_metro_stations_3_station_name_embed_2,address_metro_stations_3_station_name_embed_3,address_metro_stations_3_station_name_embed_4,address_metro_stations_3_line_name_embed_0,address_metro_stations_3_line_name_embed_1,address_metro_stations_3_line_name_embed_2,address_metro_stations_3_line_name_embed_3,address_metro_stations_3_line_name_embed_4,department_name_embed_0,department_name_embed_1,department_name_embed_2,department_name_embed_3,department_name_embed_4,category_embed_0,category_embed_1,category_embed_2,category_embed_3,category_embed_4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-0.581004,0.010968,-0.752548,0.446675,-1.263296,1.093199,0.043522,-1.025787,1.405245,-1.198341,0.088235,-0.244736,-1.194631,0.306374,0.661689,-0.680968,1.711364,0.694497,-0.831994,1.015268,1

In [23]:
X_train, X_test_val, y_train, y_test_val, = train_test_split(final_data, df['salary'], test_size=0.4, random_state=12345)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=12345)

print(f'Размеры выборок: Обучающая {X_train.shape}, Валидационная {X_test.shape}, Тестовая {X_val.shape}')

Размеры выборок: Обучающая (425714, 111), Валидационная (141905, 111), Тестовая (141905, 111)


### Случайный лес с эмбедингами

In [10]:
model_dtr = DecisionTreeRegressor(random_state=12345)

regressor = TransformedTargetRegressor(
    regressor=model_dtr,
    func=np.log,
    inverse_func=np.exp
)


param_grid = {
    'regressor__max_depth': [10, 11, 12, 13, 14, 15, 16],
    'regressor__min_samples_split': [2, 5],
    'regressor__min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=regressor,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f'Лучшие параметры: {best_params}')


y_pred = best_model.predict(X_test)
culc_metrics(y_test, y_pred)

Fitting 3 folds for each of 28 candidates, totalling 84 fits
Лучшие параметры: {'regressor__max_depth': 15, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 2}
Корень из среднеквадратичной ошибки (RMSE): 48970.264067754295
R² Score: 0.5416319993914924
Средняя абсолютная ошибка (MAE): 25268.431367611895
Средняя абсолютная процентная ошибка (SMAPE): 28.27%
Медианная абсолютная ошибка (MedAE): 14970.763403016885


Случайному лесу создание эмбедингов не принесло никакой информации и никак не улучшило обобщающую способность. Продолжем использщовать изначальный DF.

#Полносвязная нейронная сеть

Создадим свою нейронную сеть основаную на **Sequentia**
И протестируем на разных вариантах архитектур

### Полносвязная нейронная сеть с эмбедингами

In [6]:
def build_and_train_model(architecture, X_train, y_train, X_test, y_test, epochs=100, batch_size=32):
    """
    Строит и обучает модель с заданной архитектурой

    Параметры:
    architecture - список, определяющий архитектуру сети (количество нейронов в каждом слое)
    X_train, y_train - обучающие данные
    X_test, y_test - тестовые данные
    epochs - количество эпох обучения
    batch_size - размер батча

    Возвращает:
    model - обученная модель
    history - история обучения
    metrics - словарь с метриками на тестовых данных
    train_time - время обучения
    """

    input_shape = X_train.shape[1]

    model = Sequential()

    model.add(InputLayer(shape=(input_shape,)))
    model.add(Dense(architecture[0], activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    for neurons in architecture[1:]:
        model.add(Dense(neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    start_time = time.time()
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=0
    )
    train_time = time.time() - start_time

    y_pred = model.predict(X_test).flatten()

    metrics = {
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred)
    }

    return model, history, y_pred, train_time, metrics

In [24]:
architectures = {
    'small': [64, 32],
    'medium': [128, 64, 32],
    'large': [256, 128, 64, 32],
    'wide': [512, 256],
    'deep': [64, 64, 64, 64, 64]
}

results = {}

for name, arch in architectures.items():
    print(f"\nTraining {name} architecture: {arch}")
    model, history, y_pred, train_time, metrics = build_and_train_model(
        arch, X_train, y_train, X_test, y_test
    )

    results[name] = {
        'architecture': arch,
        'train_time': train_time,
        'metrics': metrics,
        'epochs_trained': len(history.history['loss'])
    }

    print(f"Training time: {train_time:.2f}s")
    culc_metrics(y_test, y_pred)


Training small architecture: [64, 32]
4435/4435 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
Training time: 1012.70s
Корень из среднеквадратичной ошибки (RMSE): 50941.297970951884
R² Score: 0.5039911649265851
Средняя абсолютная ошибка (MAE): 29103.112701003913
Средняя абсолютная процентная ошибка (SMAPE): 32.33%
Медианная абсолютная ошибка (MedAE): 20343.0625

Training medium architecture: [128, 64, 32]
4435/4435 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
Training time: 1239.07s
Корень из среднеквадратичной ошибки (RMSE): 63997.72743782658
R² Score: 0.2171500930664486
Средняя абсолютная ошибка (MAE): 29133.000331506486
Средняя абсолютная процентная ошибка (SMAPE): 31.22%
Медианная абсолютная ошибка (MedAE): 18446.55078125

Training large architecture: [256, 128, 64, 32]
4435/4435 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
Training time: 903.88s
Корень из среднеквадратичной ошибки (RMSE): 99553.35289036723
R² Score: -0.8943541865990701
Средняя абсолютная ошибка (MAE): 62541.92347013849
Средняя абсолютная процентная о

### TabNet с эмбедингами

In [24]:
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.to_numpy().reshape(-1, 1)
y_val = y_val.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1, 1)

class SMAPE(Metric):
    def __init__(self):
        self._name = "smape"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tabnet_params = {
    "n_d": 8,
    "n_a": 8,
    "n_steps": 3,
    "gamma": 1.3,
    "lambda_sparse": 1e-3,
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=2e-2),
    "mask_type": "sparsemax",
    "scheduler_params": dict(
        mode="min",
        patience=5,
        min_lr=1e-5,
        factor=0.9,
    ),
    "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
    "seed": 42,
    "verbose": 10
}

model = TabNetRegressor(**tabnet_params, device_name=device)

model.fit(
    X_train=X_train,
    y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'val'],
    eval_metric=['rmse', 'mae', SMAPE],
    max_epochs=50,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    loss_fn=torch.nn.functional.mse_loss,
)

y_pred = model.predict(X_test)

culc_metrics(y_test, y_pred)

/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 20437558003.24312| train_rmse: 141221.62524| train_mae: 85872.05342| train_smape: 184.71002| val_rmse: 115379.20418| val_mae: 85718.8759| val_smape: 184.79026|  0:00:26s
epoch 10 | loss: 10820886075.10296| train_rmse: 104044.71899| train_mae: 28615.76256| train_smape: 31.64873| val_rmse: 65343.81696| val_mae: 28682.34666| val_smape: 31.78737|  0:04:51s
epoch 20 | loss: 10630551067.8584| train_rmse: 101629.35374| train_mae: 31064.74035| train_smape: 34.09568| val_rmse: 60451.6293| val_mae: 31113.5129| val_smape: 34.13363|  0:09:12s
epoch 30 | loss: 10159129009.73706| train_rmse: 99112.52429| train_mae: 27355.95985| train_smape: 30.8259 | val_rmse: 56314.21829| val_mae: 27496.50355| val_smape: 30.97229|  0:13:24s
epoch 40 | loss: 9945627975.79911| train_rmse: 99790.60418| train_mae: 32431.17795| train_smape: 35.30642| val_rmse: 56739.63909| val_mae: 32498.53681| val_smape: 35.4043 |  0:17:36s
Stop training because you reached max_epochs = 50 with best_epoch = 36 and best

/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Корень из среднеквадратичной ошибки (RMSE): 49171.296497332296
R² Score: 0.5378608956464286
Средняя абсолютная ошибка (MAE): 26366.728262877266
Средняя абсолютная процентная ошибка (SMAPE): 29.62%
Медианная абсолютная ошибка (MedAE): 17047.4140625


#### Вывод
С эмбедингами лучше
На cpu быстрее чем на mps

### Оптимизация TabNet

In [ ]:
param_grid = {
    'n_d': [8, 16, 32],
    'n_a': [8, 16, 32],
    'n_steps': [3, 5, 7],
    'gamma': [1.0, 1.3, 1.5],
    'lambda_sparse': [0, 1e-4, 1e-3],
}

from sklearn.base import BaseEstimator, RegressorMixin

class TabNetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, lambda_sparse=1e-3):
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.lambda_sparse = lambda_sparse
        self.model = None
        
    def fit(self, X, y):
        self.model = TabNetRegressor(
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            lambda_sparse=self.lambda_sparse
        )
        self.model.fit(X, y.reshape(-1, 1))
        return self
    
    def predict(self, X):
        return self.model.predict(X).flatten()

model = TabNetWrapper()

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f'Лучшие параметры: {best_params}')


y_pred = best_model.predict(X_test)
culc_metrics(y_test, y_pred)

print("Лучшие параметры:", grid_search.best_params_)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_

epoch 0  | loss: 22581979623.97112|  0:00:11s
epoch 0  | loss: 14762814863.24909|  0:00:11s
epoch 0  | loss: 24332797271.79783|  0:00:11s
epoch 0  | loss: 24249658364.30325|  0:00:12s
epoch 0  | loss: 14640936697.53069|  0:00:11s
epoch 0  | loss: 22490206370.65704|  0:00:12s
epoch 0  | loss: 14670376605.11191|  0:00:17s
epoch 0  | loss: 22508431393.27076|  0:00:17s
epoch 0  | loss: 24246481058.65704|  0:00:18s
epoch 1  | loss: 14061122340.04332|  0:00:23s
epoch 1  | loss: 21887270586.68592|  0:00:23s
epoch 1  | loss: 23618582446.67148|  0:00:23s
epoch 1  | loss: 13482588527.82672|  0:00:24s
epoch 1  | loss: 23115953244.41878|  0:00:24s
epoch 0  | loss: 22384062560.11553|  0:00:24s
epoch 1  | loss: 21404253389.16967|  0:00:24s
epoch 0  | loss: 14667839809.61733|  0:00:24s
epoch 0  | loss: 24160459716.85198|  0:00:24s
epoch 2  | loss: 20896716777.8195|  0:00:35s
epoch 2  | loss: 13079806330.91697|  0:00:35s
epoch 2  | loss: 22499209082.91696|  0:00:36s
epoch 1  | loss: 13584044762.10831|

/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 46 | loss: 4899370672.98195|  0:19:48s
epoch 45 | loss: 14390037672.66426|  0:19:49s
[CV] END gamma=1.0, lambda_sparse=0, n_a=8, n_d=8, n_steps=3; total time=19.9min


...

epoch 39 | loss: 12231172890.80145|  2:02:13s
epoch 12 | loss: 12857714152.8953|  0:42:12s
epoch 95 | loss: 2640588691.63899|  2:43:48s
epoch 91 | loss: 12208529081.76174|  2:38:22s
epoch 0  | loss: 22495616469.48737|  0:00:14s
epoch 56 | loss: 12443616492.59206|  3:12:17s
epoch 55 | loss: 3679326064.05776|  3:03:16s
epoch 37 | loss: 13603290355.06137|  1:28:21s
epoch 36 | loss: 4442031343.82672|  2:01:40s
epoch 96 | loss: 2658311089.67509|  2:59:30s
epoch 92 | loss: 12109948117.48736|  2:54:03s
epoch 40 | loss: 12234122002.02166|  2:18:04s
epoch 1  | loss: 21308839832.49097|  0:15:55s
epoch 11 | loss: 5416475501.05415|  0:56:48s
epoch 55 | loss: 12670342925.86282|  3:18:48s
epoch 3  | loss: 18189180112.86643|  0:21:25s
epoch 13 | loss: 12801876908.82311|  0:58:11s
epoch 93 | loss: 11979478542.09386|  2:54:18s
epoch 97 | loss: 2606640854.87365|  2:59:47s
epoch 38 | loss: 13583954980.50541|  1:44:10s
epoch 2  | loss: 19806539217.79062|  0:16:10s
epoch 37 | loss: 4309468294.93141|  2:02:

/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 41 | loss: 13589721096.31768|  1:45:27s
epoch 5  | loss: 16171813301.6029|  0:22:51s
epoch 98 | loss: 11869801360.17329|  2:55:45s
epoch 59 | loss: 12466148922.68592|  3:29:36s
epoch 7  | loss: 14407270245.66065|  0:17:35s
epoch 44 | loss: 12210530482.36823|  2:19:48s
epoch 58 | loss: 3583196158.61372|  3:20:35s
epoch 0  | loss: 14698542471.8556|  0:00:19s
epoch 40 | loss: 4183046604.24549|  2:03:35s
epoch 14 | loss: 5204850863.59567|  0:58:36s
epoch 99 | loss: 11976006273.61733|  2:56:01s
epoch 42 | loss: 13386662307.58122|  1:45:50s
epoch 58 | loss: 12446655213.74729|  3:20:40s
epoch 8  | loss: 13820434924.59206|  0:17:51s
epoch 16 | loss: 12717864249.76173|  1:00:01s
epoch 45 | loss: 12175229887.30686|  2:20:14s
epoch 1  | loss: 13612809759.42238|  0:00:41s
epoch 6  | loss: 15561421656.722|  0:23:33s
epoch 9  | loss: 13474634485.37185|  0:18:11s
epoch 60 | loss: 12467580453.4296|  3:30:17s
epoch 41 | loss: 4216577907.52346|  2:04:07s
epoch 43 | loss: 13565272210.48376|  1:46:1

/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 59 | loss: 12672841761.27074|  3:36:46s
epoch 46 | loss: 12086889578.74368|  2:36:08s
epoch 17 | loss: 12682848229.19856|  1:16:10s
epoch 44 | loss: 13362935485.45848|  2:02:10s
epoch 42 | loss: 4026353401.06859|  2:20:02s
epoch 11 | loss: 12875923460.15886|  0:34:11s
epoch 3  | loss: 11038228973.51624|  0:16:51s
epoch 0  | loss: 24249889252.27437|  0:00:18s
epoch 7  | loss: 15083977372.18773|  0:39:42s
epoch 61 | loss: 12481385896.66426|  3:46:20s
epoch 60 | loss: 3567864040.66426|  3:37:17s
epoch 47 | loss: 12017149022.26716|  2:36:31s
epoch 16 | loss: 5111282849.73285|  1:15:17s
epoch 12 | loss: 12757844711.04693|  0:34:26s
epoch 45 | loss: 13303514843.03248|  2:02:33s
epoch 60 | loss: 12462542267.14802|  3:37:20s
epoch 4  | loss: 9721944246.98917|  0:17:11s
epoch 1  | loss: 23138496868.73646|  0:00:37s
epoch 18 | loss: 12605069424.7509|  1:16:43s
epoch 43 | loss: 4130493750.52708|  2:20:28s
epoch 13 | loss: 12620361319.04694|  0:34:42s
epoch 48 | loss: 11878245497.76173|  2:3

### Подготовка данных без эмбедингов

In [12]:
scaler = StandardScaler()
num_df = pd.DataFrame(scaler.fit_transform(df[num_columns]), columns=num_columns)

label_columns = []
ohe_columns = []

for column in cat_columns:
    if df[column].nunique() > 10:
        label_columns.append(column)
    else:
        ohe_columns.append(column)

to_bool = list(df[cat_columns].select_dtypes(include=['bool']).columns)
df[['salary_gross', 'employer_accredited_it_employer']] = df[['salary_gross', 'employer_accredited_it_employer']].astype(bool).astype(int)
df[to_bool] = df[to_bool].astype(int)

ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe_encoded = ohe.fit_transform(df[ohe_columns])
ohe_feature_names = ohe.get_feature_names_out(ohe_columns).tolist()
encoded_ohe_data = pd.DataFrame(ohe_encoded, columns=ohe_feature_names)

label_encoder = LabelEncoder()
for col in label_columns:
    df[col] = label_encoder.fit_transform(df[col])

X = pd.concat([df[label_columns], encoded_ohe_data, num_df], axis=1)
y = df['salary']

X_2_train, X_2_test_val, y_2_train, y_2_test_val, = train_test_split(X, y, test_size=0.4, random_state=12345)
X_2_test, X_2_val, y_2_test, y_2_val = train_test_split(X_2_test_val, y_2_test_val, test_size=0.5, random_state=12345)

print(f'Размеры выборок: Обучающая {X_2_train.shape}, Валидационная {X_2_test.shape}, Тестовая {X_2_val.shape}')

Размеры выборок: Обучающая (425714, 69), Валидационная (141905, 69), Тестовая (141905, 69)


### Полносвязная нейронная сеть без эмбедингов

In [ ]:
results_2 = {}

architectures = {
    'small': [64, 32],
    'medium': [128, 64, 32],
    'large': [256, 128, 64, 32],
    'wide': [512, 256],
    'deep': [64, 64, 64, 64, 64]
}

for name, arch in architectures.items():
    print(f"\nTraining {name} architecture: {arch}")
    model, history, y_pred, train_time, metrics = build_and_train_model(
        arch, X_2_train, y_2_train, X_2_test, y_2_test
    )

    results_2[name] = {
        'architecture': arch,
        'train_time': train_time,
        'metrics': metrics,
        'epochs_trained': len(history.history['loss'])
    }

    print(f"Training time: {train_time:.2f}s")
    culc_metrics(y_2_test, y_pred)


Training small architecture: [64, 32]
4435/4435 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
Training time: 702.78s
Корень из среднеквадратичной ошибки (RMSE): 71249.22691526295
R² Score: 0.029691776178032314
Средняя абсолютная ошибка (MAE): 41372.06422014223
Средняя абсолютная процентная ошибка (SMAPE): 45.38%
Медианная абсолютная ошибка (MedAE): 33547.609375

Training medium architecture: [128, 64, 32]
4435/4435 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
Training time: 1053.81s
Корень из среднеквадратичной ошибки (RMSE): 70796.97617631152
R² Score: 0.041970643708062916
Средняя абсолютная ошибка (MAE): 38817.90827763207
Средняя абсолютная процентная ошибка (SMAPE): 42.65%
Медианная абсолютная ошибка (MedAE): 28399.53125

Training large architecture: [256, 128, 64, 32]
4435/4435 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step
Training time: 1138.94s
Корень из среднеквадратичной ошибки (RMSE): 75041.47628239205
R² Score: -0.0763465867913884
Средняя абсолютная ошибка (MAE): 43487.600362960016
Средняя абсолютная процентная 

### TabNet без эмбедингов

In [ ]:
X_train = X_2_train.to_numpy()
X_val = X_2_val.to_numpy()
X_test = X_2_test.to_numpy()

y_train = y_2_train.to_numpy().reshape(-1, 1)
y_val = y_2_val.to_numpy().reshape(-1, 1)
y_test = y_2_test.to_numpy().reshape(-1, 1)

class SMAPE(Metric):
    def __init__(self):
        self._name = "smape"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tabnet_params = {
    "n_d": 8,
    "n_a": 8,
    "n_steps": 3,
    "gamma": 1.3,
    "lambda_sparse": 1e-3,
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=2e-2),
    "mask_type": "sparsemax",
    "scheduler_params": dict(
        mode="min",
        patience=5,
        min_lr=1e-5,
        factor=0.9,
    ),
    "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
    "seed": 42,
    "verbose": 10,
    "device_name": device
}

model = TabNetRegressor(**tabnet_params)

model.fit(
    X_train=X_train,
    y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'val'],
    eval_metric=['rmse', 'mae', SMAPE],
    max_epochs=50,
    patience=20,
    batch_size=512,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    loss_fn=torch.nn.functional.mse_loss,
    pin_memory=False
)

y_pred = model.predict(X_test)

culc_metrics(y_test, y_pred)

epoch 0  | loss: 19872069664.19348| train_rmse: 136917.37655| train_mae: 79243.44531| train_smape: 154.26673| val_rmse: 110076.88833| val_mae: 79087.0 | val_smape: 154.32972|  0:00:58s
epoch 10 | loss: 11264595244.16502| train_rmse: 106016.77754| train_mae: 29198.63867| train_smape: 32.39938| val_rmse: 68291.12645| val_mae: 29137.13281| val_smape: 32.44966|  0:10:57s
epoch 20 | loss: 11220968340.13476| train_rmse: 112126.81278| train_mae: 33202.75781| train_smape: 36.70336| val_rmse: 78464.1925| val_mae: 33103.00391| val_smape: 36.7279 |  0:20:47s
epoch 30 | loss: 11206899306.1923| train_rmse: 114423.6865| train_mae: 38514.47266| train_smape: 41.57988| val_rmse: 80557.57692| val_mae: 38373.82812| val_smape: 41.61062|  0:30:39s

Early stopping occurred at epoch 30 with best_epoch = 10 and best_val_smape = 32.44966


/Users/mandal437/hh_regression/.venv/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Корень из среднеквадратичной ошибки (RMSE): 61374.38162621274
R² Score: 0.28001469373703003
Средняя абсолютная ошибка (MAE): 29087.1015625
Средняя абсолютная процентная ошибка (SMAPE): 32.53%
Медианная абсолютная ошибка (MedAE): 19507.56640625


## Выводы